## Overview

**You will learn:** a fast and easy way to use your Developer Subscription content manager and the [ArcGIS API for Python](https://developers.arcgis.com/python/guide/) to create a data schema in the cloud and populate features for use in your apps.

With your [ArcGIS Developer Subscription](https://developers.arcgis.com/pricing/), you can use the [**Create Layer Dev Lab**](https://developers.arcgis.com/labs/data/create-a-new-dataset/) to learn how to create a new empty layer in your account.  You can then use the [ArcGIS API for Python](https://developers.arcgis.com/python/) to easily populate the layer with actual features and attributes. 

The **Create Layer** tool lets you interactively define the name, feature type (points, lines or polygons), and attribute fields for a layer, and then save it to the cloud (ArcGIS Online). This process is called [publishing a hosted feature layer](https://doc.arcgis.com/en/arcgis-online/share-maps/publish-features.htm). Behind every feature layer is a [REST feature service](https://resources.arcgis.com/en/help/arcgis-rest-api/#/Feature_Service/02r3000000z2000000/), and the ArcGIS Platform provides many tools to populate this layer with data. The [features module](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html) and [geometry module](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geometry.html) in the ArcGIS API for Python can be used with the [map widget](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.widgets.html) to interactively edit a feature layer by adding new features. Once created and populated, your apps can access this service to query, draw or edit the data.

In this lab you will create a new point dataset (feature layer) with the Create Layer tool and store it in your own developer [ArcGIS Online](https://doc.arcgis.com/en/arcgis-online/share-maps/hosted-web-layers.htm) account. You'll then use the ArcGIS API for Python to add features into the new layer. The lab will create points near Los Angeles, CA.

You will run this lab in a [Jupyter Notebook](/python/guide/using-the-jupyter-notebook-environment/) IDE that is hosted by Esri. Notebooks created here are temporary, hence you must download them if you want to preserve them for later. However, you can [install the ArcGIS API for Python](/python/guide/install-and-set-up/) locally on your computer and run this lab from it, in which case, the notebooks are stored locally on your machine.

## Steps
#### Create a connection to your Developer Subscription account

Log into ArcGIS Online by making a GIS connection to ArcGIS Online using your developer account. Replace `username` and `password` with the credentials to your ArcGIS Developer account. 
   
It is not good security practice to leave account credentials in source code shared by others, but it is beyond the scope of this lab to go over security best practices. The [getpass module](https://pymotw.com/2/getpass/) for Python provides an easy way to hide passwords in shared code.

You can use a print statement with some gis properties to verify sign-in.

In [ ]:
import getpass
from arcgis.gis import *

password = getpass.getpass("Please enter password: ")
dev_gis = GIS('https://www.arcgis.com', 'johnnyDev', password)
print("Successfully logged in to {} as {}".format(dev_gis.properties.urlKey + '.' + dev_gis.properties.customBaseUrl,
                                                 dev_gis.users.me.username))

#### Search for the empty feature layer

You can use the ArcGIS API for Python [Helper Objects](https://developers.arcgis.com/python/guide/using-the-GIS/#Helper-objects) to manage GIS resources. One such helper object is the [Content Manager](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#contentmanager). You use its [search method](https://developers.arcgis.com/python/guide/accessing-and-creating-your-content/#Searching-for-content) to retrieve items from your GIS. The ArcGIS API for Python Guide also documents the process of [searching your gis for feature layers](https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/#Searching-the-GIS-for-feature-layers).

The ArcGIS API for Python is a front end to the [ArcGIS REST API](https://developers.arcgis.com/documentation/core-concepts/rest-api/). The full [search reference](http://resources.arcgis.com/en/help/arcgis-rest-api/#/Search_reference/02r3000000mn000000/) documentation details how to search for items, including proper use of wildcards for characters and phrases. You'll use a wildcard to search for the empty feature layer created in the first portion of this lab (Steps 1-8).

The `search()` method returns a list of `Item` objects that match the search criteria. 

In [ ]:
flyr_search_results = dev_gis.content.search(query="""title: "Griffith*" AND 
                                         type: "Feature Service" """, 
                                         max_items=10)
flyr_search_results

#### Retrieve the feature layer object from the search results

Since the search returns a list, you'll use Python [indexing](https://docs.python.org/3/tutorial/introduction.html#lists) to retrieve the object you want. In this case, the item returned is a feature layer collection item, so you'll need to obtain the list of layers in the collection and use indexing to access the layer you want.

In [ ]:
flyr_collection_item = flyr_search_results[0]
flayers = flyr_collection_item.layers
flayer = flayers[0]
flayer.properties.name

#### Define a function to serve as the event listener for the map widget on_click event

You'll use a Python function to add fetures to the feature layer for locations you click on the map. Further illustrations of concepts and operations are available at [using the geometry service and geometry functions](https://developers.arcgis.com/python/sample-notebooks/using-geometry-functions/) in the [ArcGIS API for Python Guide](https://developers.arcgis.com/python/guide/). The API Reference documents the map widget [draw method](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.widgets.html#arcgis.widgets.MapView.draw) that you'll use to initially place graphic markers at the locations where you'll add features to the layer.

The function creates a feature from the location clicked on the map. The **`OBJECTID`** attribute **MUST** be included in the attributes dictionary parameter to create a valid feature. You can also add additional attributes from the layer schema if desired.

In [ ]:
from arcgis import geometry
from arcgis import features

def create_feature(map1, g):
    try:
        oid = 1
        pt = geometry.Point(g)
        feat = features.Feature(geometry=pt, attributes={'OBJECTID': oid})
        flayer.edit_features(adds=[feat])
        print(str(g))
        map1.draw(g)
        oid += 1
    except:
        print("Couldn't create the feature. Try again, please...")

#### Assign a map object to a variable. Center the map in Los Angeles with a zoom level of your choice.

In [ ]:
map1 = dev_gis.map('Los Angeles', 10)
map1

#### Execute the on_click event to begin adding locations to the map

Use the function you defined above as the callback parameter for the map widget [on_click](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.widgets.html#arcgis.widgets.MapView.on_click) event. Click on the map to draw a graphic marker for each feature you want to add to the layer. The callback also outputs a string representation of the geometry.

In [ ]:
map1.on_click(create_feature)

#### Clear the graphic markers from the map

You'll clear the graphics so you can subsequently add the layer to the map to verify features have been added.

In [ ]:
map1.clear_graphics()

#### Use the map widget `add_layer` method to add the feature layer to the map

You should see the locations you clicked in the previous steps now visualized as features in the feature layer, with symbology defined by the Create Layer tool.

In [ ]:
map1.add_layer(flayer)